# Exercício 3 - Trabalho Prático 2

**Grupo 6:** 


Ruben Silva - pg57900

Luís Costa - pg55970

# Problema: 

1. Usando a experiência obtida na resolução dos problemas 1 e 2, e usando, ao invés  do grupo abeliano multiplicativo $\,\mathbb{F}_p^\ast\,$,  o  grupo abeliano aditivo que usou na pergunta 2,   
    1. Construa ambas as versões  IND-CPA segura e IND-CCA segura do esquema de cifra ElGamal em curvas elípticas.
    2. Construa uma implementação em curvas elípticas de um protocolo autenticado de “Oblivious Transfer” $\,\kappa$-out-of-$n\,$.

## Parte I - IND-CPA

[Paper ElGamal Elliptic Curve](https://www.ams.org/journals/mcom/1987-48-177/S0025-5718-1987-0866109-5/S0025-5718-1987-0866109-5.pdf)

In [112]:
from sage.all import *
from cryptography.hazmat.primitives import hashes
import secrets
import hashlib
import base64
import random

## Class Edwards22519

1. É a class do exercício 2 sem a parte da "criptografia". Contendo primordialmente as funções das curvas elipticas

In [113]:
class Edwards25519():
    #Private Functions
    def __init__(self):
        self.p = 2**255 - 19
        self.d = -121665 * inverse_mod(121666, self.p) % self.p
        self.q = 2**252 + 27742317777372353535851937790883648493
        

        self.gy = 4 * self._modp_inv(5) % self.p
        self.gx = self._recover_x(self.gy, 0)
        self.G = (self.gx, self.gy, 1, self.gx * self.gy % self.p)

    def _sha512(self, data):
        return hashlib.sha512(data).digest()
    
    def _sha512_modq(self, data):
        return int.from_bytes(self._sha512(data), 'little') % self.q
    
    def _point_add(self, P, Q):
        A, B = (P[1]-P[0]) * (Q[1]-Q[0]) % self.p, (P[1]+P[0]) * (Q[1]+Q[0]) % self.p
        C, D = 2 * P[3] * Q[3] * self.d % self.p, 2 * P[2] * Q[2] % self.p
        E, F, G, H = B-A, D-C, D+C, B+A
        return (E*F, G*H, F*G, E*H)
    
    def _point_sub(self, P, Q):
        A, B = (P[1]-P[0]) * (Q[1]+Q[0]) % self.p, (P[1]+P[0]) * (Q[1]-Q[0]) % self.p
        C, D = 2 * P[3] * Q[2] * self.d % self.p, 2 * P[2] * Q[3] % self.p
        E, F, G, H = B-A, D-C, D+C, B+A
        return (E*F, G*H, F*G, E*H)

    def _point_mult(self,s,P):
        Q = (0, 1, 1, 0)
        while s > 0:
            if s & 1:
                Q = self._point_add(Q, P)
            P = self._point_add(P, P)
            s >>= 1
        return Q
    
    def _point_equal(self,P,Q):
        if (P[0] * Q[2] - Q[0] * P[2]) % self.p != 0:
            return False
        if (P[1] * Q[2] - Q[1] * P[2]) % self.p != 0:
            return False
        return True
    

    def _mod_sqrt(self):
        return pow(2, (self.p-1)//4, self.p)
    
    def _modp_inv(self, x):
        return pow(x, self.p-2, self.p)
    
    def _recover_x(self, y, sign):
        if y >= self.p:
            return None
        x2 = (y*y-1) * self._modp_inv(self.d*y*y+1)
        if x2 == 0:
            if sign:
                return None
            else:
                return 0
        x = pow(x2, (self.p+3) // 8, self.p)
        if (x*x - x2) % self.p != 0:
            x = x * self._mod_sqrt() % self.p
        if (x*x - x2) % self.p != 0:
            return None
        if (x & 1) != sign:
            x = self.p - x
        return x
    
    def _point_compress(self, P):
        zinv = self._modp_inv(P[2])
        x = P[0] * zinv % self.p
        y = P[1] * zinv % self.p
        return int(y | ((x & 1) << 255)).to_bytes(32, 'little')
    
    def _point_decompress(self, s):
        if len(s) != 32:
            raise Exception("Invalid input length for decompression")
        y = int.from_bytes(s, "little")
        sign = y >> 255
        y &= (1 << 255) - 1
        x = self._recover_x(y, sign)

        if x is None:
            return None
        else:
            return (x, y, 1, x*y % self.p)
        

    def _secret_expand(self, secret):
        if len(secret) != 32:
            raise Exception("Invalid input length for secret key")
        h = self._sha512(secret)
        a = int.from_bytes(h[:32], "little")
        a &= (1 << 254) - 8
        a |= (1 << 254)
        return (a, h[32:])
    

## ElGamal em Curva Eliptica IND-CPA

1. É herdado as propriedades da Curva `Edwards25519` com o _init_

2. É definido a função **ElGamal_GenKeys** que retorna a `public_key` e a `private_key`
    1. É gerado a `private_key` tal que $0<$ private_key $<q$ reduzido ao módulo de $q$
    2. É gerado a `public_key` tal que $s.G$
        1. $s$->private_key
        2. $G$->ponto base da curva
    3. a public_key é comprimida e é devolvido o par "sk,pk" como esperado de um ElGamal

3. É definido a função **ElGamal_Enc** que retorna o `ciphertext` $(R,C)$
    1. A public_key é descomprimida para obter $P = s.G$
    2. É escolhido um valor descartável aleatório entre $0$ e $q$
    3. É Calculado $R = k.G$ e $kP = k.P$
        1. $R = k.G$ equivale $g^k$ em ElGamal do exercício 1
        2. $kP = k.P$ equivale $(g^s)^k$ em ElGamal do exercício 1
    4. É cifrada a mensagem apartir da soma dos pontos mantendo assim a estrutura de um `grupo abeliano aditivo` resultando no R
    5. Retorna $R$ e $C$ comprimidos em tuplo

3. É definido a função **ElGamal_Dec** que retorna o `ponto na curva decifrado`
    1. A $R$ e $C$ são descomprimidos
    2. É calculado o ponto $S=s.R=s.(k.G)$ usando a private_key $s$ 
    3. É invertido o eixo $x$ de S para facilitar na "adição" (obtermos assim a subtração para recuperarmos a mensagem)
    4. Devolve o ponto decifrado

In [114]:
class EC_ElGamal_CPA(Edwards25519):
    def __init__(self):
        super().__init__()

    def ElGamal_GenKeys(self):
        private_key = int.from_bytes(os.urandom(32), "little") % self.q
        public_key = self._point_mult(private_key, self.G)
        return private_key, self._point_compress(public_key)

    def ElGamal_Enc(self, public_key, message_point):
        pub_point = self._point_decompress(public_key)
        if pub_point is None:
            raise ValueError("Invalid public key")

        k = int.from_bytes(os.urandom(32), "little") % self.q  
        R = self._point_mult(k, self.G)  
        kP = self._point_mult(k, pub_point)  
        C = self._point_add(message_point, kP) 

        return self._point_compress(R), self._point_compress(C)

    def ElGamal_Dec(self, private_key, R_compressed, C_compressed):
        R = self._point_decompress(R_compressed)
        C = self._point_decompress(C_compressed)
        if R is None or C is None:
            raise ValueError("Invalid ciphertext")

        S = self._point_mult(private_key, R)  
        S_neg = (-S[0] % self.p, S[1], S[2], -S[3] % self.p)  # Negação (-x, y)
        M = self._point_add(C, S_neg)
        return M


## ElGamal em Curva Eliptica IND-CCA

1. É herdado as propriedades da Curva `Edwards25519` com o _init_

2. É definido a função **ElGamal_GenKeys_CCA** que retorna a `public_key` e a `private_key`
    1. É gerado a `private_key` tal que $0<$ private_key $<q$ reduzido ao módulo de $q$
    2. É gerado a `public_key` tal que $s.G$
        1. $s$->private_key
        2. $G$->ponto base da curva
    3. a public_key é comprimida e é devolvido o par "sk,pk" como esperado de um ElGamal

3. É definido a função **ElGamal_Enc_CCA** que retorna o `ciphertext` $(R,C)$
    1. A public_key é descomprimida para obter $P = s.G$
    2. É escolhido um valor descartável aleatório $r$ entre $0$ e $q$
    3. É obtido $k$ derivando $r$ numa hash
    4. É Calculado $R = k.G$ e $kP = k.P$
        1. $R = k.G$ equivale $g^k$ em ElGamal do exercício 1
        2. $kP = k.P$ equivale $(g^s)^k$ em ElGamal do exercício 1
    5. É cifrada a mensagem apartir da soma dos pontos resultando no R
    6. É gerar a `tag` com um hash de $(R||C||r)$
    6. Retorna $R$ e $C$ comprimidos em tuplo

3. É definido a função **ElGamal_Dec_CCA** que retorna o `ponto na curva decifrado`
    1. A $R$ e $C$ são descomprimidos
    2. É calculado o ponto $S=s.R=s.(k.G)$ usando a private_key $s$ 
    3. É invertido o eixo $x$ de S para facilitar na "adição" (obtermos assim a subtração para recuperarmos a mensagem)
    4. É verificado a "tag" para detetar algum ataque à `integridade da mensagem`
    5. Devolve o ponto decifrado se não estiver afetado

In [115]:
class EC_ElGamal_CCA(Edwards25519):
    def __init__(self):
        super().__init__()
        self.H = lambda x: hashlib.sha256(x).digest()
        self.G_point = self.G 

    def ElGamal_GenKeys_CCA(self):
        private_key = int.from_bytes(os.urandom(32), "little") % self.q
        public_key = self._point_mult(private_key, self.G_point)
        return private_key, self._point_compress(public_key)

    def ElGamal_Enc_CCA(self, public_key, message_point):
        pub_point = self._point_decompress(public_key)
        if pub_point is None:
            raise ValueError("Invalid public key")

        r = os.urandom(32)
        k_bytes = self.H(r)
        k = int.from_bytes(k_bytes, "little") % self.q

        R = self._point_mult(k, self.G_point)
        kP = self._point_mult(k, pub_point)
        C = self._point_add(message_point, kP)

        # Gerar a tag
        #hash(R || C || r)
        tag_input = self._point_compress(R)+ self._point_compress(C) + r
        tag = self.H(tag_input)

        return self._point_compress(R), self._point_compress(C), r, tag

    def ElGamal_Dec_CCA(self, private_key, R_compressed, C_compressed, r, tag):
        R = self._point_decompress(R_compressed)
        C = self._point_decompress(C_compressed)
        if R is None or C is None:
            raise ValueError("Invalid ciphertext")

        S = self._point_mult(private_key, R)
        S_neg = (-S[0] % self.p, S[1], S[2], -S[3] % self.p)
        M = self._point_add(C, S_neg)

        # Verificar a tag
        #hash(R || C || r) (inverso da cifragem)
        tag_input = self.H(R_compressed + self._point_compress(C) + r)

        if tag_input != tag:
            raise ValueError("Ciphertext integrity check failed (CCA security)")

        return M

## Oblivious Transfer em Curva Elíptica ElGamal

1. É herdado as propriedades da Curva `Edwards25519` com o _init_
    1. É definido $n$ e $\kappa$ segundo o protocolo
    2. É Criado algumas hashs especificas para o protocolo ser mantido "fiel"
2. É definido a função **ElGamal_GenKeys_OT** que retorna o `seed`, `lista de chaves privadas`, `lista de chaves publicas` e `tag tau`
    1. É gerado uma `seed` secreta aleatória com 32 bytes
    2. É Calculado uma "tag" $\tau$ tal que $H(seed||I)$ 
    3. Para cada índice i em I:
        1. $private\_key_i = H(seed || i) \mod q$ 
        2. $public\_key_i = private\_key_i . G$
    4. Preenchimento da lista
        1. Preenche $public\_key_i [i]$ com chaves públicas para $i \in I$
        2. Para $i \notin I$, gera chaves aleatorias tambme dentro de $G$

3. É definido a função **ElGamal_Enc_OT** que retorna `n ciphertexts`$
    1. É gerado um valor $r$ aleatório com 32 bytes
    2. É mascarado a mensagem tal que $y = m_i \oplus G(r)$
    3. É calculado $r' = H(r||y||\tau)$ e convertido no escalar $k$
    4. É calculado $R = k.G$ (similar ao $g^k$ do ElGamal Tradicional)
    5. É descomprimido $public\_key_i [i]$ para se obter $P_i = private\_key_i . G$
    6. É Calculado $kP = k.P_i$ (similar ao $(g^s)^k$)
    7. É mapeado o valor $M = y.G$
    8. Cifrar o ponto $C = M + kP$
    9. Gerar a verficação $c= F(r||r')$
    10. Retonar n ciphertexts

4. É definido a função **ElGamal_Dec_OT** que retorna o `n msns decifradas`
    1. É feito o inverso do anterior retornando as mensagens decifradas

In [116]:
class ObliviousTransferKofN(Edwards25519):
    def __init__(self, n, k):
        super().__init__()
        self.n = n  # nr Mensagens
        self.k = k  # nr Mensagens pedidas
        self.H = lambda x: hashlib.sha256(x).digest()  # Hash for key derivation and tag
        self.G_hash = lambda x: hashlib.sha256(x + b"g").digest()  # Fog
        self.F = lambda x: hashlib.sha256(x + b"f").digest()  # Randomness check

    # Receiver: Gerar as keys
    def ElGamal_GenKeys_OT(self, I):
        seed = os.urandom(32)  # Secret seed
        tau = self.H(seed + bytes(sorted(I)))  # Authentication tag tau: hash(I, s)

        # Gerar k chaves privadas e respetivas chaves públicas
        sk = {}
        pk = [None] * self.n
        for i, idx in enumerate(sorted(I), 1):
            seed = self.H(seed + i.to_bytes(4, "little"))
            sk[idx] = int.from_bytes(seed, "little") % self.q
            pk[idx] = self._point_compress(self._point_mult(sk[idx], self.G))

        # Encher o resto do array com chaves públicas aleatórias
        for j in range(self.n):
            if pk[j] is None:
                rand_sk = int.from_bytes(os.urandom(32), "little") % self.q
                pk[j] = self._point_compress(self._point_mult(rand_sk, self.G))

        return seed, sk, pk, tau

    # Provider: Cifrar as mensagens
    def ElGamal_Enc_OT(self, p, messages, tau):
        assert len(messages) == self.n, "Number of messages must match n"
        ciphertexts = []
        p_points = [self._point_decompress(pk) for pk in p]

        for i in range(self.n):
            m = messages[i]
            r = os.urandom(32)  # Randomness
            y = bytes(a ^ b for a, b in zip(m, self.G_hash(r)))  # XOR FOG
            r_prime = self.H(r + y + tau)  # Tau randomness
            k = int.from_bytes(r_prime, "little") % self.q
            R = self._point_mult(k, self.G)
            kP = self._point_mult(k, p_points[i])
            M_point = self._point_mult(int.from_bytes(y, "little") % self.q, self.G)
            C = self._point_add(M_point, kP)
            c = self.F(r + r_prime)  # Consistency check
            ciphertexts.append((y, self._point_compress(R), self._point_compress(C), c, r))  # Inclui r

        return ciphertexts
    
    def ElGamal_Dec_OT(self, sk, ciphertexts, tau):
    
        assert len(ciphertexts) == self.n, "Number of ciphertexts must match n"
        decrypted_messages = {}

        for idx in sk.keys():  # Apenas os índices em I têm chaves privadas
            y, R_compressed, C_compressed, c, r = ciphertexts[idx]
            R = self._point_decompress(R_compressed)
            C = self._point_decompress(C_compressed)
            if R is None or C is None:
                raise ValueError(f"Invalid ciphertext for index {idx}")

            # Calcula S = sk[idx] * R
            S = self._point_mult(sk[idx], R)
            S_neg = (-S[0] % self.p, S[1], S[2], -S[3] % self.p)  # Negação do ponto S
            M = self._point_add(C, S_neg)  # Recupera o ponto M = C - S

            # Verifica se M corresponde a y * G
            y_recovered = int.from_bytes(y, "little") % self.q
            M_expected = self._point_mult(y_recovered, self.G)
            if not self._point_equal(M, M_expected):
                raise ValueError(f"Decryption failed for index {idx}: point mismatch")

            # Verifica consistência com c
            r_prime = self.H(r + y + tau)
            if c != self.F(r + r_prime):
                raise ValueError(f"Consistency check failed for index {idx}")

            # Recupera a mensagem original: m = y ⊕ G_hash(r)
            m = bytes(a ^ b for a, b in zip(y, self.G_hash(r)))
            decrypted_messages[idx] = m

        return decrypted_messages


### Correr e Testar as Classes acima

1. ElGamal CPA

2. ElGamal CCA

3. ElGamal em Oblivious Transfer

In [117]:
def test_ElGamalCPA():
    ec_elgamal = EC_ElGamal_CPA()

    # Gerar as keys
    priv_key, pub_key = ec_elgamal.ElGamal_GenKeys()
    print(f"Private Key: {priv_key}")
    print(f"Public Key: {pub_key.hex()}")

    # cifrar a mensagem
    message_point = ec_elgamal.G 
    R, C = ec_elgamal.ElGamal_Enc(pub_key, message_point)
    print(f"Ciphertext (R, C): {R.hex()}, {C.hex()}")

    # decifrar a mensagem
    decrypted_message = ec_elgamal.ElGamal_Dec(priv_key, R, C)
    print(f"Decrypted Message: {decrypted_message}")
    print(f"MEssage Point: {message_point}")
    assert ec_elgamal._point_equal(message_point, decrypted_message)
    print("Decryption successful")

def test_ElGamalCCA():
    ec_elgamal = EC_ElGamal_CCA()

    # gerar as chaves
    priv_key, pub_key = ec_elgamal.ElGamal_GenKeys_CCA()
    print(f"Private Key: {priv_key}")
    print(f"Public Key: {pub_key.hex()}")

    # cifrar a mensagem
    message_point = ec_elgamal.G  
    R, C, r, tag = ec_elgamal.ElGamal_Enc_CCA(pub_key, message_point)
    print(f"Ciphertext (R, C, r, tag): {R.hex()}, {C.hex()}, {r.hex()}, {tag.hex()}")

    # decifrar a mensagem
    decrypted_message = ec_elgamal.ElGamal_Dec_CCA(priv_key, R, C, r, tag)
    print(f"Decrypted Message: {decrypted_message}")
    print(f"MEssage Point: {message_point}")
    assert ec_elgamal._point_equal(message_point, decrypted_message)
    print("Decryption successful")

def test_ElGamalCCA_OT():
    ot = ObliviousTransferKofN(4, 2)

    # Receiver: gerar as keys e autenticação
    s, sk, pk, tau = ot.ElGamal_GenKeys_OT([0, 2])
    print(f"Secret Seed: {s.hex()}")
    print(f"Private Keys: {sk}")
    print(f"Public Keys: {[pk_i.hex() for pk_i in pk]}")
    print(f"Authentication Tag: {tau.hex()}")

    # Provider: ciphertexts
    messages = [b"Hello", b"World", b"Foo", b"Bar"]
    ciphertexts = ot.ElGamal_Enc_OT(pk, messages, tau)
    for i, (y, R, C, c, r) in enumerate(ciphertexts):
        print(f"Message {i}: {messages[i]}")
        print(f"Ciphertext (y, R, C, c, r): {y.hex()}, {R.hex()}, {C.hex()}, {c.hex()}, {r.hex()}")

    # Receiver: decifrar as mensagens
    decrypted_messages = ot.ElGamal_Dec_OT(sk, ciphertexts, tau)
    for i, m in decrypted_messages.items():
        print(f"Decrypted Message {i}: {m}")
        assert m == messages[i], f"Decrypted message {m} does not match original {messages[i]}"
    print("Decryption successful")

In [118]:
print("=== Testes ===")
print("CPA ->")
test_ElGamalCPA()
print("===")
print("CCA ->")
test_ElGamalCCA()
print("===")
print("CCA OT ->")
test_ElGamalCCA_OT()

=== Testes ===
CPA ->
Private Key: 3364999846320620934847484416811922348008489871330056161386286238717908889663
Public Key: 214fed5ee81e36f8ba1e0803efbbe4f7de10313531191c38114640a6cb40b128
Ciphertext (R, C): 133500b0307c23289ef66b504c676809a67bb990b9e023f4adbe665d6746fbd8, 9bbd3d7f8d0227abc91a545e3fb7a9a2f247bb93bfac7b6543a1034df4627b1a
Decrypted Message: (-256341229311468359481510727788760774949256101980455350474722714342155350852078258437759269720168713633602149729895098282941492266780407080358175057481900, 3626445620888563101103915082993682218757622566870512357408457414702675193944708059660763657813484343855672282546394729965737715201666560116092191180454784, -1475100390988514547748281752927695042416666325140693319611820415362241734890953378355023893175126375220379447028347884913581042681756568641903501184729600, 630199499755270237692991074328671249515866378777733683056918756108024013566865885453186444610864870089094573474561404420771409141312666750517916358738301)
MEssage Point: (1